In [4]:
import pandas as pd

# Load Excel
df = pd.read_excel("department_marketing_mix_data.xlsx")

# Combine relevant text fields (e.g., Channel + Department + Year) into a single string
df["text"] = df.apply(lambda row: f"{row['Department']} department ran {row['Channel']} campaign in {row['Year']}. Spend: {row['Spend']}, ROI: {row['ROI']}, Incremental ROI: {row['Incremental ROI']}", axis=1)


In [5]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)


c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 7/7 [00:00<00:00,  7.55it/s]


In [6]:
import faiss
import numpy as np

# Convert to numpy float32
embedding_matrix = np.array(embeddings).astype("float32")

# Build FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # L2 = Euclidean distance
index.add(embedding_matrix)

# Optional: Save the index
faiss.write_index(index, "faiss_mmm_index.index")


In [7]:
query = "Digital campaigns in 2023 with high ROI"
query_embedding = model.encode([query]).astype("float32")

# Search top 5 similar entries
distances, indices = index.search(query_embedding, k=5)

# Print results
for i in indices[0]:
    print(df.iloc[i]["text"])


Digital department ran Social Media campaign in 2022. Spend: 42705.7, ROI: 1.93, Incremental ROI: 1.68
Digital department ran Content Marketing campaign in 2023. Spend: 44610.26, ROI: 2.02, Incremental ROI: 1.58
Digital department ran Social Media campaign in 2023. Spend: 41307.43, ROI: 2.12, Incremental ROI: 1.91
Digital department ran Social Media campaign in 2024. Spend: 29335.73, ROI: 0.66, Incremental ROI: 0.14
Digital department ran Content Marketing campaign in 2022. Spend: 39964.12, ROI: 1.72, Incremental ROI: 1.22


In [ ]:
!pip install langchain-groq

In [13]:
from langchain_groq import ChatGroq 

In [ ]:
!pip install langchain-ollama

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [40]:
llm = OllamaLLM(model="deepseek-r1:1.5b")

In [19]:
# Load FAISS index
index = faiss.read_index("faiss_mmm_index.index")

# Load metadata (your Excel)
df = pd.read_excel("department_marketing_mix_data.xlsx")

# Load sentence embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [22]:
documents = df.apply(lambda row: f"In {row['Year']}, the ROI for {row['Channel']} in {row['Department']} was {row['ROI']}.", axis=1).tolist()


In [24]:
import pickle

In [25]:
# Save metadata
with open("metadata.pkl", "wb") as f:
    pickle.dump(documents, f)

In [26]:
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

In [28]:

# Function to get top-k documents
def retrieve_context(query, k=3):
    query_embedding = embedding_model.encode([query])
    D, I = index.search(query_embedding, k)
    return [metadata[i] for i in I[0]]

In [ ]:
def ask_deepseek(prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def agentic_rag_chat():
    chat_history = ""
    while True:
        user_query = input("User: ")

        context = retrieve_context(user_query)
        if not any(context):
            follow_up = "I couldn't find enough information. Can you tell me more? For example, which department or year?"
            print(f"Agent: {follow_up}")
            clarification = input("User: ")
            context = retrieve_context(clarification + " " + user_query)

        context_str = "\n".join(context)
        prompt = f"Context:\n{context_str}\n\nQuestion: {user_query}\nAnswer:"
        response = ask_deepseek(prompt)
        print(f"Agent: {response}")


####generate code

In [38]:
# Step 2: Load Data and FAISS Index
df = pd.read_excel("department_marketing_mix_data.xlsx")
df["text"] = df.apply(lambda row: f"{row['Department']} department ran {row['Channel']} campaign in {row['Year']}. Spend: {row['Spend']}, ROI: {row['ROI']}, Incremental ROI: {row['Incremental ROI']}", axis=1)

# Load embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
index = faiss.read_index("faiss_mmm_index.index")


In [ ]:
def retrieve_similar_documents(query, top_k=5):
    query_embedding = embed_model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, k=top_k)
    
    docs = []
    for idx in indices[0]:
        text = df.iloc[idx]["text"]
        docs.append(Document(page_content=text))
    return docs

# Step 4: Define DeepSeek Model + Prompt Template
LANGUAGE_MODEL = OllamaLLM(model="deepseek-r1:1.5b")

PROMPT_TEMPLATE = """
You are a strategic pharma marketing assistant. Use the provided context to help with marketing tactic planning.
If the context is insufficient to answer the query confidently, say so and ask the user a clarification question.
Limit your response to 3 sentences max. Be concise and data-driven.

Query: {user_query} 
Context: {document_context} 
Answer:
"""

In [41]:
# Step 5: Main Chat Loop (Agentic Behavior)
def start_agentic_chat():
    print("📊 Pharma Tactic Planner - Agentic RAG Chatbot (DeepSeek R1)")
    print("Type 'exit' to end the conversation.\n")
    
    chat_history = []
    while True:
        user_query = input("🧠 You: ")
        if user_query.lower() == "exit":
            print("👋 Goodbye!")
            break

        docs = retrieve_similar_documents(user_query, top_k=5)

        if not docs or len(docs) < 2:
            print("🤔 Not enough relevant data found. Can you provide more context or rephrase your question?")
            continue

        answer = generate_answer(user_query, docs)
        print(f"🤖 DeepSeek: {answer}")

        # Store in history for potential use later
        chat_history.append({"user": user_query, "bot": answer})


In [42]:
# Step 6: Start the RAG Chat
start_agentic_chat()


📊 Pharma Tactic Planner - Agentic RAG Chatbot (DeepSeek R1)
Type 'exit' to end the conversation.

🤖 DeepSeek: <think>
Okay, let me try to figure this out. The user provided some data from an HR department's recruitment portals campaign across several years and their associated costs and ROI. They want a marketing tactic plan based on this context. 

First, I notice the data spans from 2021 to 2024, with spending increasing each year but ROI also fluctuating. The incremental ROI shows how much value the new campaigns added compared to previous ones. So, the company's campaigns seem profitable when viewed in isolation and had varying returns when compared over time.

The user might be looking for a strategy to enhance future campaigns or compare past efforts. They likely want actionable steps based on this data. I should break down each year's performance and see where the company can leverage these findings.

They could benefit from expanding the campaign, adjusting strategies based on 